# Choice of Test Prompts Matching and Mismatching the Linguistic features of the Train Set 

## Preparations

In [3]:
# installations and downloads
!pip install matplotlib numpy pandas

Traceback (most recent call last):
  File "c:\Users\line\.conda\envs\myenv1\lib\runpy.py", line 187, in _run_module_as_main
    mod_name, mod_spec, code = _get_module_details(mod_name, _Error)
  File "c:\Users\line\.conda\envs\myenv1\lib\runpy.py", line 146, in _get_module_details
    return _get_module_details(pkg_main_name, error)
  File "c:\Users\line\.conda\envs\myenv1\lib\runpy.py", line 110, in _get_module_details
    __import__(pkg_name)
  File "c:\Users\line\.conda\envs\myenv1\lib\site-packages\spacy\__init__.py", line 15, in <module>
    from .cli.info import info  # noqa: F401
  File "c:\Users\line\.conda\envs\myenv1\lib\site-packages\spacy\cli\__init__.py", line 3, in <module>
    from ._util import app, setup_cli  # noqa: F401
  File "c:\Users\line\.conda\envs\myenv1\lib\site-packages\spacy\cli\_util.py", line 9, in <module>
    import typer
  File "c:\Users\line\.conda\envs\myenv1\lib\site-packages\typer\__init__.py", line 12, in <module>
    from click.termui import get_t

In [1]:
# imports
from datasets import load_dataset
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from collections import Counter
from ast import literal_eval
from scipy.stats import wilcoxon

## Load Datasets with Analysis (Train and Test Set)

In [2]:
analysis_train = pd.read_csv('train.csv', converters = {"lemmata":literal_eval, "pos_tags":literal_eval})
analysis_test = pd.read_csv('test.csv', converters = {"lemmata":literal_eval, "pos_tags":literal_eval})

In [3]:
analysis_train.head()

,prompt_id,sentence,tokens,lemmata,pos_tags,sent_len,sentiment_pos,sentiment_neg,sentiment_neutr
0,0,SUBREDDIT: r/relationships\nTITLE: I (f/22) ha...,"['relationship', 'f/22', 'figure', 'want', 'kn...","[relationship, f/22, figure, want, know, girl,...","[NOUN, NOUN, NOUN, AUX, VERB, NOUN, VERB, ADJ,...",49,0.902425,0.097561,0.000014
1,0,Backstory:\n,['backstory'],[backstory],[NOUN],3,0.425992,0.281691,0.292318
2,0,When I (f/22) went through my first real break...,"['f/22', 'go', 'real', 'breakup', '2', 'year',...","[f/22, go, real, breakup, 2, year, ago, need, ...","[NOUN, VERB, ADJ, NOUN, NUM, NOUN, ADV, VERB, ...",33,0.734857,0.077453,0.187689
3,0,It was a horrible time in my life due to livin...,"['horrible', 'time', 'life', 'live', 'mother',...","[horrible, time, life, live, mother, finally, ...","[ADJ, NOUN, NOUN, ADJ, NOUN, ADV, VERB, NOUN, ...",27,0.359685,0.616026,0.024289
4,0,I can admit because of it was an emotional wre...,"['admit', 'emotional', 'wreck', 'guy', 'stable...","[admit, emotional, wreck, guy, stable, know, d...","[VERB, ADJ, NOUN, NOUN, ADJ, PROPN, NOUN]",25,0.933862,0.045779,0.020359


In [4]:
analysis_test.head()

,prompt_id,sentence,tokens,lemmata,pos_tags,sent_len,sentiment_pos,sentiment_neg,sentiment_neutr
0,0,SUBREDDIT: r/relationships\nTITLE: Me,['relationship'],[relationship],[NOUN],9,0.277909,0.462360,0.259731
1,0,"[19 F] with my friend [19 M], not sure if I ma...","['19', 'f', 'friend', '19', 'm', 'sure', 'mess...","[19, f, friend, 19, m, sure, mess, thing]","[NUM, PROPN, NOUN, NUM, NOUN, ADJ, NOUN, NOUN]",24,0.992242,0.003785,0.003973
2,0,POST:,[],[],[],2,0.209116,0.246918,0.543966
3,0,Hello hello everybody.,"['hello', 'hello', 'everybody']","[hello, hello, everybody]","[INTJ, INTJ, PRON]",4,0.478715,0.091491,0.429794
4,0,I hope this isn't too trivial of a question to...,"['hope', 'trivial', 'question', 'ask', 'feel',...","[hope, trivial, question, ask, feel, bit, dept...","[VERB, ADJ, NOUN, INTJ, VERB, NOUN, NOUN, VERB...",55,0.817698,0.116945,0.065357


## Analysis of Sentence Length and Sentiment Score (Train and Test Set)

In [6]:
# Sentence Length and Sentiment Scores | get mean for each prompt (group all sentences in prompt by id)

# Sentence Length
df_sent_len_avg = analysis_train.groupby('prompt_id', as_index=False)['sent_len'].mean()

# Sentiment Scores
df_sentim_pos = analysis_train.groupby('prompt_id', as_index=False)['sentiment_pos'].mean()
df_sentim_neg = analysis_train.groupby('prompt_id', as_index=False)['sentiment_neg'].mean()
df_sentim_neutr = analysis_train.groupby('prompt_id', as_index=False)['sentiment_neutr'].mean()

# df with all of the above features
df_sentence_sentim_train = pd.concat([df_sent_len_avg, df_sentim_pos, df_sentim_neg, df_sentim_neutr], axis = 1)
df_sentence_sentim_train = df_sentence_sentim_train.loc[:,~df_sentence_sentim_train.columns.duplicated()] # removes duplicate column 'prompt_id'
df_sentence_sentim_train

,prompt_id,sent_len,sentiment_pos,sentiment_neg,sentiment_neutr
0,0,18.636364,0.568697,0.242274,0.189029
1,1,17.800000,0.365290,0.227349,0.407360
2,2,19.071429,0.388516,0.262108,0.349376
3,3,20.333333,0.529530,0.178965,0.291505
4,4,17.100000,0.520360,0.254645,0.224995
...,...,...,...,...,...
116717,116717,21.866667,0.674410,0.205873,0.119717
116718,116718,15.700000,0.583203,0.199618,0.217179
116719,116719,18.727273,0.362161,0.147906,0.489934
116720,116720,18.857143,0.525659,0.192807,0.281534


In [7]:
# Sentence Length and Sentiment Scores | get MEAN for each prompt (group all sentences in prompt by id, then average inside these groups)

# Sentence Length
df_sent_len_avg = analysis_test.groupby('prompt_id', as_index=False)['sent_len'].mean()

# Sentiment Scores
df_sentim_pos = analysis_test.groupby('prompt_id', as_index=False)['sentiment_pos'].mean()
df_sentim_neg = analysis_test.groupby('prompt_id', as_index=False)['sentiment_neg'].mean()
df_sentim_neutr = analysis_test.groupby('prompt_id', as_index=False)['sentiment_neutr'].mean()

# als beispiel hier mal in einem data frame, reicht ja aber auch nur die groupby() oben fürs indexen unserer matches
df_sentence_sentim_test = pd.concat([df_sent_len_avg, df_sentim_pos, df_sentim_neg, df_sentim_neutr], axis = 1)
df_sentence_sentim_test = df_sentence_sentim_test.loc[:,~df_sentence_sentim_test.columns.duplicated()] # removes duplicate column 'prompt_id'
df_sentence_sentim_test

,prompt_id,sent_len,sentiment_pos,sentiment_neg,sentiment_neutr
0,0,20.166667,0.501960,0.258953,0.239086
1,1,22.250000,0.438876,0.376221,0.184903
2,2,25.428571,0.511979,0.357651,0.130370
3,3,24.642857,0.521677,0.263471,0.214852
4,4,14.846154,0.435765,0.231189,0.333046
...,...,...,...,...,...
6548,6548,14.038462,0.364716,0.249390,0.385894
6549,6549,19.583333,0.483704,0.251235,0.265061
6550,6550,20.000000,0.488279,0.204835,0.306886
6551,6551,23.900000,0.465454,0.371708,0.162838


## Analysis of POS Tags and Lemmata (Train and Test Set)

### Train Set

In [8]:
# POS over train set

# list of POS  per prompt
grouped_pos_train = analysis_train.groupby('prompt_id', as_index=True, group_keys=True)['pos_tags']
grouped_pos_train = grouped_pos_train.aggregate(np.sum)

# combine in one list
all_pos = [word for sublist in grouped_pos_train.values.tolist() for word in sublist]

# count occurences
pos_counts = Counter(all_pos)

pos_most_common_train = pos_counts.most_common(20)
pos_most_common_train

[('NOUN', 5192107),
 ('VERB', 2633892),
 ('ADJ', 1498375),
 ('ADV', 702404),
 ('PROPN', 700458),
 ('NUM', 359161),
 ('ADP', 146147),
 ('INTJ', 111983),
 ('AUX', 85941),
 ('X', 28690),
 ('PART', 24913),
 ('PRON', 18478),
 ('SCONJ', 17247),
 ('SYM', 14814),
 ('PUNCT', 7906),
 ('CCONJ', 5959),
 ('DET', 1373),
 ('SPACE', 838)]

In [9]:
df_pos_train = pd.DataFrame()

# count how often the 20 most common POS overall appear in each prompt
for tag, count in pos_most_common_train:
    pos_counts_per_prompt = [Counter(pos_list)[tag] for pos_list in grouped_pos_train]
    pos_counts_per_prompt = pd.DataFrame(pos_counts_per_prompt)
    df_pos_train = pd.concat([df_pos_train, pos_counts_per_prompt], ignore_index=True, axis=1)

# df with most commmon POS count per prompt
tags = [tag for tag, count in pos_most_common_train]
df_pos_train = df_pos_train.rename(columns = dict(zip(list(range(20)), tags)))
df_pos_train

,NOUN,VERB,ADJ,ADV,PROPN,NUM,ADP,INTJ,AUX,X,PART,PRON,SCONJ,SYM,PUNCT,CCONJ,DET,SPACE
0,59,32,18,5,5,2,0,0,2,0,0,0,0,0,0,0,0,0
1,36,13,15,5,11,8,1,0,0,0,0,0,0,0,0,0,0,0
2,34,15,13,6,1,4,3,0,0,0,0,0,0,0,0,0,0,0
3,44,12,12,2,0,11,0,0,0,0,0,0,0,9,0,0,0,0
4,39,27,21,11,3,1,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116717,30,19,8,9,11,0,1,1,1,0,1,0,0,0,0,1,0,0
116718,36,28,15,7,5,1,2,0,1,0,0,0,0,0,0,0,0,0
116719,37,15,12,7,12,0,1,1,0,0,0,0,0,0,1,0,0,0
116720,38,15,14,2,2,1,1,3,1,2,0,0,0,0,0,0,0,0


In [10]:
# Lemmata Over whole Train Set

# list of lemmata per prompt
grouped_lem_train = analysis_train.groupby('prompt_id', as_index=True, group_keys=True)['lemmata']
grouped_lem_train = grouped_lem_train.aggregate(np.sum)

# combine in one list
all_words = [word for sublist in grouped_lem_train.values.tolist() for word in sublist]

# count occurences
lem_counts = Counter(all_words)

lem_most_common_train = lem_counts.most_common(20)
lem_most_common_train

[('like', 153035),
 ('want', 150789),
 ('relationship', 148660),
 ('know', 138289),
 ('feel', 132815),
 ('time', 124852),
 ('friend', 120156),
 ('year', 116369),
 ('go', 110327),
 ('think', 105160),
 ('tell', 98899),
 ('get', 97189),
 ('say', 91495),
 ('thing', 85025),
 ('month', 77595),
 ('talk', 77078),
 ('m', 73680),
 ('work', 71137),
 ('start', 66237),
 ('good', 65098)]

In [11]:
df_lem_train = pd.DataFrame()

# count how often the 20 most common lemmata overall appear in each prompt
for tag, count in lem_most_common_train:
    lem_counts_per_prompt = [Counter(lem_list)[tag] for lem_list in grouped_lem_train]
    lem_counts_per_prompt = pd.DataFrame(lem_counts_per_prompt)
    df_lem_train = pd.concat([df_lem_train, lem_counts_per_prompt], ignore_index=True, axis=1)

# df with most commmon lemmata count per prompt
tags = [tag for tag, count in lem_most_common_train]
df_lem_train = df_lem_train.rename(columns = dict(zip(list(range(20)), tags)))
df_lem_train

,like,want,relationship,know,feel,time,friend,year,go,think,tell,get,say,thing,month,talk,m,work,start,good
0,0,2,1,3,0,3,3,3,3,3,0,2,0,0,1,1,0,0,0,1
1,1,0,0,2,3,1,0,0,0,0,0,1,1,1,1,0,0,0,0,0
2,1,1,3,1,0,0,1,0,2,0,2,0,0,0,1,0,1,1,0,0
3,0,0,0,0,0,1,0,2,0,0,0,0,0,0,4,0,0,0,0,1
4,0,1,1,0,2,2,0,0,1,0,0,0,0,1,1,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116717,0,2,3,5,0,1,5,1,0,0,1,0,0,0,0,0,0,0,0,2
116718,2,4,4,5,4,0,1,1,1,3,0,1,0,0,0,0,1,0,2,1
116719,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1
116720,2,1,2,0,1,1,0,1,1,2,0,1,0,2,1,0,1,0,2,1


### Test Set

In [12]:
# POS Over Test Prompts

# list of POS  per prompt
grouped_pos_test = analysis_test.groupby('prompt_id', as_index=True, group_keys=True)['pos_tags']
grouped_pos_test = grouped_pos_test.aggregate(np.sum)

df_pos_test = pd.DataFrame()

# count how often the 20 most common POS in train set appear in each test prompt
for tag, count in pos_most_common_train:
    pos_counts_per_prompt = [Counter(pos_list)[tag] for pos_list in grouped_pos_test]
    pos_counts_per_prompt = pd.DataFrame(pos_counts_per_prompt)
    df_pos_test = pd.concat([df_pos_test, pos_counts_per_prompt], ignore_index=True, axis=1)

# rename df with most commmon POS count per prompt
tags = [tag for tag, count in pos_most_common_train]
df_pos_test = df_pos_test.rename(columns = dict(zip(list(range(20)), tags)))
df_pos_test

,NOUN,VERB,ADJ,ADV,PROPN,NUM,ADP,INTJ,AUX,X,PART,PRON,SCONJ,SYM,PUNCT,CCONJ,DET,SPACE
0,56,33,11,14,9,2,1,7,3,2,0,1,0,0,0,0,0,0
1,32,17,7,8,8,1,1,1,0,1,0,0,0,0,0,0,0,0
2,25,8,7,3,7,6,0,0,1,0,0,0,0,0,1,0,0,0
3,48,23,18,14,8,0,1,0,0,0,0,0,0,0,0,0,0,0
4,31,11,6,6,9,3,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6548,55,20,13,6,1,4,2,0,2,0,0,0,0,0,0,0,0,0
6549,37,22,10,5,3,5,0,2,1,1,0,0,1,0,0,0,0,0
6550,21,17,5,2,0,2,1,0,0,0,0,0,0,0,0,0,0,0
6551,62,36,16,13,5,9,1,1,0,1,0,0,0,0,0,0,0,0


In [13]:
# Lemmata Over Test Prompts

# list of lemmata per prompt
grouped_lem_test = analysis_test.groupby('prompt_id', as_index=True, group_keys=True)['lemmata']
grouped_lem_test = grouped_lem_test.aggregate(np.sum)

df_lem_test = pd.DataFrame()

# count how often the 20 most common lemmata in train set appear in each prompt
for tag, count in lem_most_common_train:
    lem_counts_per_prompt = [Counter(lem_list)[tag] for lem_list in grouped_lem_test]
    lem_counts_per_prompt = pd.DataFrame(lem_counts_per_prompt)
    df_lem_test = pd.concat([df_lem_test, lem_counts_per_prompt], ignore_index=True, axis=1)

# df with most commmon lemmata count per prompt
tags = [tag for tag, count in lem_most_common_train]
df_lem_test = df_lem_test.rename(columns = dict(zip(list(range(20)), tags)))
df_lem_test

,like,want,relationship,know,feel,time,friend,year,go,think,tell,get,say,thing,month,talk,m,work,start,good
0,1,3,3,1,1,2,2,0,3,1,0,1,1,3,0,3,1,1,1,0
1,1,4,1,0,0,2,3,1,0,0,0,0,1,1,0,1,0,0,0,0
2,0,0,2,1,0,2,3,0,0,0,0,1,0,0,1,0,2,0,0,0
3,1,0,0,0,0,0,0,1,2,0,0,1,0,0,0,0,0,0,0,0
4,0,2,1,0,0,0,0,1,0,1,0,0,0,0,2,0,1,2,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6548,2,2,3,0,2,2,0,5,1,0,0,0,0,0,0,0,1,1,2,2
6549,1,0,0,1,1,0,0,0,1,0,0,1,0,0,1,0,0,0,2,0
6550,0,0,2,1,2,1,0,0,0,3,0,1,0,0,1,0,1,0,0,0
6551,1,0,2,0,6,1,0,3,3,0,3,2,1,0,0,0,1,0,1,1


## Select Matching and Mismatching Test Prompts

### Sentence Length

In [14]:
# defining criteria from main training set statistics

# we choose test prompts with statistics that fall into/out of the 95% confidence interval of the train statistics: [mean +/- 1.96 * std]
x = 1.96

In [15]:
# Sentence Length
crit_sent_len_upper = df_sentence_sentim_train['sent_len'].mean() + x * df_sentence_sentim_train['sent_len'].std()
crit_sent_len_lower = df_sentence_sentim_train['sent_len'].mean() - x * df_sentence_sentim_train['sent_len'].std()

In [16]:
# select IDs for Sentence Length matches and mismatches
matches_sent_len = df_sentence_sentim_test.loc[(df_sentence_sentim_test['sent_len'] >= crit_sent_len_lower) & (df_sentence_sentim_test['sent_len'] <= crit_sent_len_upper)]
mismatches_sent_len = df_sentence_sentim_test.loc[(df_sentence_sentim_test['sent_len'] < crit_sent_len_lower) | (df_sentence_sentim_test['sent_len'] > crit_sent_len_upper)]

match_id_sentlen = np.random.choice(matches_sent_len['prompt_id'], 13)
mismatch_id_sentlen = np.random.choice(mismatches_sent_len['prompt_id'], 13)

### Sentiment Score

In [17]:
# Sentiment Score
crit_sentim_pos_upper = df_sentence_sentim_train['sentiment_pos'].mean() + x * df_sentence_sentim_train['sentiment_pos'].std()
crit_sentim_pos_lower = df_sentence_sentim_train['sentiment_pos'].mean() - x * df_sentence_sentim_train['sentiment_pos'].std()

# Sentiment Score
crit_sentim_neg_upper = df_sentence_sentim_train['sentiment_neg'].mean() + x * df_sentence_sentim_train['sentiment_neg'].std()
crit_sentim_neg_lower = df_sentence_sentim_train['sentiment_neg'].mean() - x * df_sentence_sentim_train['sentiment_neg'].std()

# Sentiment Score
crit_sentim_neutr_upper = df_sentence_sentim_train['sentiment_neutr'].mean() + x * df_sentence_sentim_train['sentiment_neutr'].std()
crit_sentim_neutr_lower = df_sentence_sentim_train['sentiment_neutr'].mean() - x * df_sentence_sentim_train['sentiment_neutr'].std()

In [18]:
# select IDs for Sentiment matches and mismatches
matches_sentim = df_sentence_sentim_test.loc[(df_sentence_sentim_test['sentiment_pos'] >= crit_sentim_pos_lower) & (df_sentence_sentim_test['sentiment_pos'] <= crit_sentim_pos_upper) &
                                     (df_sentence_sentim_test['sentiment_neg'] >= crit_sentim_neg_lower) & (df_sentence_sentim_test['sentiment_neg'] <= crit_sentim_neg_upper) &
                                     (df_sentence_sentim_test['sentiment_neutr'] >= crit_sentim_neutr_lower) & (df_sentence_sentim_test['sentiment_neutr'] <= crit_sentim_neutr_upper)]
mismatches_sentim = df_sentence_sentim_test.loc[(df_sentence_sentim_test['sentiment_pos'] < crit_sentim_pos_lower) | (df_sentence_sentim_test['sentiment_pos'] > crit_sentim_pos_upper) &
                                     (df_sentence_sentim_test['sentiment_neg'] < crit_sentim_neg_lower) | (df_sentence_sentim_test['sentiment_neg'] > crit_sentim_neg_upper) &
                                     (df_sentence_sentim_test['sentiment_neutr'] < crit_sentim_neutr_lower) | (df_sentence_sentim_test['sentiment_neutr'] > crit_sentim_neutr_upper)]


match_id_sentim = np.random.choice(matches_sentim['prompt_id'], 13)
mismatch_id_sentim = np.random.choice(mismatches_sentim['prompt_id'], 13)

### Lemmata

In [19]:
# Lemmata: check significant difference between test prompt lemmata and most common train lemmata

mismatch_lem = []
match_lem = []

for i in range(len(df_lem_test)):
    d = df_lem_test.loc[i].values - df_lem_train.mean()
    stat, p = wilcoxon(d)
    if p < 0.05:
        mismatch_lem.append(i)
    else:
        match_lem.append(i)

In [20]:
# select IDs for Lemmata matches and mismatches
match_id_lem = np.random.choice(match_lem, 13)
mismatch_id_lem = np.random.choice(mismatch_lem, 13)

### POS Tags

In [21]:
# POS Tags: check significant difference between test prompt POS and most common train POS

mismatch_pos = []
match_pos = []

for i in range(len(df_pos_test)):
    d = df_pos_test.loc[i].values - df_pos_train.mean()
    stat, p = wilcoxon(d)
    if p < 0.05:
        mismatch_pos.append(i)
    else:
        match_pos.append(i)

In [22]:
# select IDs for POS matches and mismatches
match_id_pos = np.random.choice(match_pos, 13)
mismatch_id_pos = np.random.choice(mismatch_pos, 13)

### Create list of matching and mismatching IDs

In [23]:
# helper function to flatten list of matches
def flatten_extend(matrix):
    flat_list = []
    for row in matrix:
        flat_list.extend(row)
    return flat_list

In [24]:
# prepare list of matching and mismatching ids
match_ids = [match_id_sentlen, match_id_sentim, match_id_lem, match_id_pos]
mismatch_ids = [mismatch_id_sentlen, mismatch_id_sentim, mismatch_id_lem, mismatch_id_pos]

match_ids = flatten_extend(match_ids)
mismatch_ids = flatten_extend(mismatch_ids)

match_ids = np.unique(match_ids)
mismatch_ids = np.unique(mismatch_ids)

### Load dataset by CarperAI

In [ ]:
# load the dataset
dataset = load_dataset("CarperAI/openai_summarize_tldr")
dataset

In [ ]:
# split test and train in dataframes with prompt and label column
train = pd.DataFrame(dataset['train'])
test = pd.DataFrame(dataset['test'])

prompts_train = train['prompt']
prompts_test = test['prompt']

### Select and save test prompts via id

In [125]:
# find test prompts with matching ids and save them
match_prompts = prompts_test[match_ids]
match_prompts.to_csv("match_prompts.csv")

In [126]:
# find test prompts with mismatching ids and save them
mismatch_prompts = prompts_test[mismatch_ids]
mismatch_prompts.to_csv("mismatch_prompts.csv")